In [1]:
import numpy as np
import cantera as ct
from cantera import ctml_writer 

In [2]:
convert_data = ctml_writer.convert('cti_one.cti')

In [3]:
type(convert_data)

NoneType

In [35]:
gas = ct.Solution('cti_one.cti')

In [36]:
gas.TPX = 300, 101325, 'PLIGC:1.7, PLIGH:0.26, PLIGO:0.3'

In [37]:
gas()


  ligpy:

       temperature             300  K
          pressure          101325  Pa
           density         13.4938  kg/m^3
  mean mol. weight         332.179  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy               0                0     J
   internal energy           -7509       -2.494e+06     J
           entropy          18.297             6078     J/K
    Gibbs function         -5489.2       -1.823e+06     J
 heat capacity c_p               0                0     J/K
 heat capacity c_v          -25.03            -8314     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
             PLIGH       0.115044          0.15116         -2.16244
             PLIGO       0.132743         0.168792         -2.01934
             PLIGC       0.752212         0.680048        -0.284737
     [  +91 minor]        

In [41]:
def ignitionDelay(df, species):
    """
    This function computes the ignition delay from the occurence of the
    peak in species' concentration.
    """
    return df[species].argmax()

In [42]:
import time
import pandas as pd

r = ct.IdealGasReactor(contents=gas, name='Batch Reactor')
reactorNetwork = ct.ReactorNet([r])

# now compile a list of all variables for which we will store data
stateVariableNames = [r.component_name(item) for item in range(r.n_vars)]

# use the above list to create a DataFrame
timeHistory = pd.DataFrame(columns=stateVariableNames)

t0 = time.time()

# This is a starting estimate. If you do not get an ignition within this time, increase it
estimatedIgnitionDelayTime = 0.1
t = 0

counter = 1;
while(t < estimatedIgnitionDelayTime):
    t = reactorNetwork.step()
    if (counter%10 == 0):
        # We will save only every 10th value. Otherwise, this takes too long
        # Note that the species concentrations are mass fractions
        timeHistory.loc[t] = reactorNetwork.get_state()
    counter+=1

# We will use the 'oh' species to compute the ignition delay
tau = ignitionDelay(timeHistory, 'oh')

#Toc
t1 = time.time()

print('Computed Ignition Delay: {:.3e} seconds. Took {:3.2f}s to compute'.format(tau, t1-t0))

# If you want to save all the data - molefractions, temperature, pressure, etc
# uncomment the next line
# timeHistory.to_csv("time_history.csv")

KeyError: 'oh'